# Evaluation different models and parameters for vessels segmentation

In [2]:
import sys
sys.path.append('..')

In [3]:
import os  
import random
import cv2
import numpy as np
from matplotlib import pyplot as plt
from segmentation_models.backbones import get_preprocessing
from keras import backend as K

from classes.model_utils import ModelUtils
from classes.image_utils import ImageUtils
from classes.process_results import ProcessResults
from classes.metrics import Metrics

## Initialize

In [5]:
val_frame_path = '/home/andrea/learning/vessel_segmentation/custom_train/val_imgs/val'
val_mask_path = '/home/andrea/learning/vessel_segmentation/custom_train/val_masks/val'
img_height = 384
img_width = 384

image_ut = ImageUtils(val_frame_path, val_mask_path, img_height, img_width, architecture='unet')

In [4]:
from scipy.signal import medfilt2d
from skimage import data, color
from skimage import measure

## Load models

In [ ]:
prefix = '/home/andrea/learning/vessel_segmentation/result_code/UNET/models/'

In [ ]:
linknet_resnet_100 =  ModelUtils.load_model(os.path.join(prefix, 'model_linknet_disk_pretrainedRESNET18_100noeqa'))
linknet_resnet_150 =  ModelUtils.load_model(os.path.join(prefix, '/model_linknet_disk_pretrainedRESNET18_150noeqa'))
linknet_resnet31_100 =  ModelUtils.load_model(os.path.join(prefix, '/model_linknet_disk_pretrainedRESNET31_100noeqa'))
linknet_vgg19_100 =  ModelUtils.load_model(os.path.join(prefix, '/model_linknet_disk_pretrainedVGG_100noeqa'))
psp_resnet18_100 =  ModelUtils.load_model(os.path.join(prefix, 'model_pspnet_disk_pretrainedRESNET18_100noeqa'))
psp_resnet18_100SGD =  ModelUtils.load_model(os.path.join(prefix, 'model_pspnet_disk_pretrainedRESNET18_SGD100noeqa'))
unet_resnet18_100 =  ModelUtils.load_model(os.path.join(prefix, 'model_unet_disk_pretrainedRESNET18_100noeqa'))
unet_resnet18_200 =  ModelUtils.load_model(os.path.join(prefix, 'model_unet_disk_pretrainedRESNET18_200noeqa'))
unet_resnet34_200 =  ModelUtils.load_model(os.path.join(prefix, 'model_unet_disk_pretrainedRESNET34_200noeqa'))

## PSP

In [ ]:
process_results_pspnet = ProcessResults()

process_results_pspnet.predict_images(backbone='resnet18', 
                                      val_frame_path=val_frame_path,
                                      val_mask_path=val_mask_path,
                                      img_height=img_height,
                                      img_width=img_width,
                                      model=psp_resnet18_100,
                                      preprocess=True)

In [ ]:
masks_psp = process_results_pspnet.tresholdibg(process_results_pspnet._masks)

filtered_images_psp = []
for im in process_results_pspnet._predicted_imgs:
    filt_img = process_results_pspnet.med_ext_med_filter(im)
    filtered_images_psp.append(np.squeeze(process_results_pspnet.tresholding(filt_img)))

In [ ]:
sensitivity_psp = Metrics.sensitivity(np.array(masks_psp), np.array(filtered_images_psp))
specifity_psp = Metrics.specificity(np.array(masks_psp), np.array(filtered_images_psp))
dice_psp = Metrics.dice_coef(np.array(np.array(masks_psp), np.array(filtered_images_psp))

In [ ]:
print('PSP sensitivity: {}'. format(sensitivity_psp))
print('PSP specifity: {}'. format(specifity_psp))
print('PSP dice coefficient: {}'. format(dice_psp))

## Linknet

In [ ]:
process_results_linknet = ProcessResults()

process_results_linknet.predict_images(backbone='resnet18', 
                                       val_frame_path=val_frame_path,
                                       val_mask_path=val_mask_path, 
                                       img_height=img_height, 
                                       img_width=img_width, 
                                       model=linknet_resnet_100, 
                                       preprocess=True)

In [ ]:
masks_link = process_results_linknet.tresholdibg(process_results_linknet._masks)

filtered_images_link = []
for im in process_results_linknet._predicted_imgs:
    filt_img = process_results_linknet.med_ext_med_filter(im)
    filtered_images_link.append(np.squeeze(process_results_linknet.tresholding(filt_img)))

In [ ]:
sensitivity_link = Metrics.sensitivity(np.array(masks_link), np.array(filtered_images_link))
specifity_link = Metrics.specificity(np.array(masks_link), np.array(filtered_images_link))
dice_link = Metrics.dice_coef(np.array(np.array(masks_link), np.array(filtered_images_link))

In [ ]:
print('LinkNet sensitivity: {}'. format(sensitivity_link))
print('LinkNet specifity: {}'. format(specifity_link))
print('LinkNet dice coefficient: {}'. format(dice_link))

## Unet

In [ ]:
￼process_results_unet = ProcessResults()

process_results_unet.predict_images(backbone='resnet18',  
                                    val_frame_path=val_frame_path,
                                    val_mask_path=val_mask_path,
                                    img_height=img_height,
                                    img_width=img_width,
                                    model=unet_resnet18_100,
                                    preprocess=True)

In [ ]:
masks_unet = process_results_unet.tresholdibg(process_results_unet._masks)

filtered_images_unet = []
for im in process_results_unet._predicted_imgs:
    filt_img = process_results_unet.med_ext_med_filter(im)
    filtered_images_unet.append(np.squeeze(process_results_unet.tresholding(filt_img)))

In [ ]:
sensitivity_unet = Metrics.sensitivity(np.array(masks_unet), np.array(filtered_images_unet))
specifity_unet = Metrics.specificity(np.array(masks_unet), np.array(filtered_images_unet))
dice_unet = Metrics.dice_coef(np.array(np.array(masks_unet), np.array(filtered_images_unet))

In [ ]:
print('Unet sensitivity: {}'. format(sensitivity_unet))
print('Unet specifity: {}'. format(specifity_unet))
print('Unet dice coefficient: {}'. format(dice_unet))